In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import glob
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Conv3D,MaxPooling3D,Dropout,Flatten,LeakyReLU,BatchNormalization,Conv2D,MaxPooling2D
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm
import zipfile


In [0]:
comofod='http://www.vcl.fer.hr/comofod/comofod_small.rar'
zip_dir = tf.keras.utils.get_file('comofod_small.rar', origin=comofod)
zip_dir_base = os.path.dirname(zip_dir)
!pip install patool
import patoolib
os.chdir(zip_dir_base)
patoolib.extract_archive("comofod_small.rar", outdir=zip_dir_base)

In [0]:
auimg    = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*_O_*')
auimg=auimg[4620:]#for 180 authentic
#auimg=auimg[4260:] for 540 authentic
jpeg     = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_JC*')
jpeg=jpeg[1260:]#540 jpeg compressed images
blur     = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_IB*')
blur=blur[420:]#180 blurred images
noise    = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_NA*')
noise=noise[420:]
bright   = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_BC*')
bright=bright[420:]
color    = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_CR*')
color=color[420:]
contrast = glob.glob(zip_dir_base + '/CoMoFoD_small_v2/*F_CA*')
contrast=contrast[420:]
print(len(auimg))
print(len(jpeg))
print(len(blur))


In [0]:
#Test1 for jpeg compressed
test   = zip_dir_base + '/Test'
os.makedirs(test)
target = zip_dir_base + '/Test/Authentic'
os.makedirs(target)
target2=zip_dir_base + '/Test/Jpeg'
os.makedirs(target2)

for i in auimg:
  shutil.move(i, target)
  
for i in jpeg:
  shutil.move(i, target2)

In [0]:
#Test2 for blurred after runtime reset
test   = zip_dir_base + '/Test'
os.makedirs(test)
target = zip_dir_base + '/Test/Authentic'
os.makedirs(target)
target3=zip_dir_base + '/Test/Blur'
os.makedirs(target3)

for i in auimg:
  shutil.move(i, target)
  
for i in blur:
  shutil.move(i, target3)
print(len(blur))

In [0]:
#Test3 for noise runtime reset not required henceforth
shutil.rmtree(target3)
target4=zip_dir_base + '/Test/Noise'
os.makedirs(target4)

for i in noise:
  shutil.move(i, target4)
print(len(noise))

In [0]:
#Test4 for brightness change
shutil.rmtree(target4)
target5=zip_dir_base + '/Test/Bright'
os.makedirs(target5)

for i in bright:
  shutil.move(i, target5)
print(len(bright))

In [0]:
#Test5 for color reduction
shutil.rmtree(target5)
target6=zip_dir_base + '/Test/Color'
os.makedirs(target6)

for i in color:
  shutil.move(i, target6)
print(len(color))

In [0]:
#Test6 for contrast adjustment
shutil.rmtree(target6)
target7=zip_dir_base + '/Test/Contrast'
os.makedirs(target7)

for i in contrast:
  shutil.move(i, target7)
print(len(contrast))

In [0]:
#to be run for all tests
test_dir = os.path.join(zip_dir_base, 'Test')

def mfr(im):

  
  medi=cv2.medianBlur(im,5)
  medi2=cv2.medianBlur(medi,5)
  medi=cv2.subtract(medi2, medi)
  medi=medi[..., np.newaxis]
  
  avg = cv2.blur(im,(5,5))
  avg = cv2.subtract(avg,im)
  avg = avg[..., np.newaxis]
  
  k=np.array([[0,0,0,0,0], [0,-1,2,-1,0], [0,2,-4,2,0], [0,-1,2,-1,0], [0,0,0,0,0]], np.int32)
  k1=k/4
  srm1=cv2.filter2D(im, -1, k1)
  srm1 = srm1[...,np.newaxis]
  
  k=np.array([[-1,2,-2,2,-1], [2,-6,8,-6,2], [-2,8,-12,8,-2], [2,-6,8,-6,2], [-1,2,-2,2,-1]], np.int32)
  k2=k/12
  srm2=cv2.filter2D(im, -1, k2)
  srm2 = srm2[...,np.newaxis]
  
  k=np.array([[0,0,0,0,0], [0,0,0,0,0], [0,1,-2,1,0], [0,0,0,0,0], [0,0,0,0,0]], np.int32)
  k3=k/2
  srm3=cv2.filter2D(im, -1, k3)
  srm3=srm3[...,np.newaxis]
  
  imge=np.concatenate((medi,avg), axis=2)
  imge=np.concatenate((imge,srm1), axis=2)
  imge=np.concatenate((imge,srm2), axis=2)
  imge=np.concatenate((imge,srm3), axis=2)
  
  imge=imge[..., np.newaxis]
  return imge

test_image_generator = ImageDataGenerator(preprocessing_function=mfr,
                                          featurewise_center=True,
                                          featurewise_std_normalization=True)#rescale=1./255,
test_batches = test_image_generator.flow_from_directory( shuffle=False,
                                                         directory=test_dir, 
                                                         target_size=(128,128,5),
                                                         class_mode='binary',
                                                         color_mode='grayscale')

In [0]:
#model architecture
ini=tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False)

model=Sequential()
model.add(Conv3D(64,(1,1,5), strides=1,  activation='relu', input_shape=(128,128,5,1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D((3,3), strides=2, padding='same'))

model.add(Conv3D(64,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Conv3D(128,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

#model.add(LeakyReLU())
model.add(Conv3D(256,(3,3,1),padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))
#model.add(LeakyReLU())

model.add(Conv3D(256,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Conv3D(512,(3,3,1),padding='same', activation='relu'))
model.add(BatchNormalization())
#model.add(LeakyReLU())
model.add(MaxPooling3D((3,3,1),padding='same', strides=2))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

#model.add(Dropout(0.5))
#model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

#model.add(Dropout(0.5))
#model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer=ini, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.001), activation='relu'))

model.add(Dense(2, activation='softmax'))

In [0]:
#model compile and weights adding
from google.colab import drive
drive.mount('/content/drive')



model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.load_weights('trained_model_weights')

In [0]:
print(model.evaluate_generator(test_batches))# for accuracy
y_pred= np.argmax(model.predict_generator(test_batches), axis=1)
labels= test_batches.labels


In [0]:
#for precision,recall and f1
from sklearn.metrics import precision_score, recall_score
precision = precision_score(list(labels), list(y_pred), pos_label=1, average='binary')
recall    = recall_score(list(labels), list(y_pred), pos_label=1, average='binary')
f1        = 2*recall*precision/(recall+precision)
print(precision)
print(recall)
print(f1)